In [0]:
# Mount the District container
storageAccount = "gen10datafund2205"
storageContainer = "capstone-group3-raw"
clientSecret = "-ZS8Q~NwOKfwEpVOg3Teb1pPtxDbz616XjlXLbuU"
clientid = "2ca50102-5717-4373-b796-39d06568588d"
mount_point = "/mnt/capstone-group3-raw"
    
    
configs = {"fs.azure.account.auth.type": "OAuth",
   "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
   "fs.azure.account.oauth2.client.id": clientid,
   "fs.azure.account.oauth2.client.secret": clientSecret,
   "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/d46b54b2-a652-420b-aa5a-2ef7f8fc706e/oauth2/token",
   "fs.azure.createRemoteFileSystemDuringInitialization": "true"}

try:
    dbutils.fs.unmount(mount_point)
except:
    pass


dbutils.fs.mount(
source = f"abfss://{storageContainer}@{storageAccount}.dfs.core.windows.net/", 
mount_point = mount_point, 
extra_configs = configs)

/mnt/capstone-group3-raw has been unmounted.
Out[3]: True

In [0]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np

In [0]:
%fs
ls /mnt/capstone-group3-raw/

path,name,size,modificationTime
dbfs:/mnt/capstone-group3-raw/ACSST5Y2020.S1901_dataset.csv,ACSST5Y2020.S1901_dataset.csv,1520328,1659984095000
dbfs:/mnt/capstone-group3-raw/DistrictDataCleaned.csv,DistrictDataCleaned.csv,48613,1659732238000
dbfs:/mnt/capstone-group3-raw/FederallyFundedDistricts.csv,FederallyFundedDistricts.csv,510945,1659754753000
dbfs:/mnt/capstone-group3-raw/NYCSchoolDistricts.csv,NYCSchoolDistricts.csv,347923,1659751742000
dbfs:/mnt/capstone-group3-raw/csvfiles/,csvfiles/,0,0
dbfs:/mnt/capstone-group3-raw/district-householdincome.csv,district-householdincome.csv,29882,1659730056000
dbfs:/mnt/capstone-group3-raw/districtdata.csv/,districtdata.csv/,0,1659751605000
dbfs:/mnt/capstone-group3-raw/districtdatafinal.csv,districtdatafinal.csv,50790,1660315269000
dbfs:/mnt/capstone-group3-raw/districtdatafinalcleaning.csv/,districtdatafinalcleaning.csv/,0,1659835886000
dbfs:/mnt/capstone-group3-raw/districtdatamedianincomerevised.csv/,districtdatamedianincomerevised.csv/,0,1659792537000


In [0]:
df = pd.read_csv('/dbfs/mnt/capstone-group3-raw/districtdatafinal.csv')

In [0]:
df.columns

Out[6]: Index(['DistrictName', 'GraduationPercentageRate', 'DropoutPercentageRate',
 'FundingPerStudent', 'TotalEnrolled', 'MalesEnrolled',
 'FemalesEnrolled', 'Percentage of Minority Students', 'HouseholdIncome',
 'FederallyFunded'],
 dtype='object')

In [0]:
df.rename(columns = {'Percentage of Minority Students':'Minority'}, inplace = True)

In [0]:
df.corr()

Out[8]:

,GraduationPercentageRate,DropoutPercentageRate,FundingPerStudent,TotalEnrolled,MalesEnrolled,FemalesEnrolled,Minority,HouseholdIncome
GraduationPercentageRate,1.000000,-0.789692,-0.026112,-0.212696,-0.216245,-0.207854,-0.359994,0.344243
DropoutPercentageRate,-0.789692,1.000000,-0.065178,0.053425,0.055943,0.050488,0.172194,-0.367961
FundingPerStudent,-0.026112,-0.065178,1.000000,-0.163294,-0.164037,-0.161670,0.119716,0.230662
TotalEnrolled,-0.212696,0.053425,-0.163294,1.000000,0.997646,0.997367,0.538394,0.056430
MalesEnrolled,-0.216245,0.055943,-0.164037,0.997646,1.000000,0.990046,0.541824,0.055165
FemalesEnrolled,-0.207854,0.050488,-0.161670,0.997367,0.990046,1.000000,0.532010,0.057685
Minority,-0.359994,0.172194,0.119716,0.538394,0.541824,0.532010,1.000000,0.335914
HouseholdIncome,0.344243,-0.367961,0.230662,0.056430,0.055165,0.057685,0.335914,1.000000


Federal Funds T-Test Analysis

In [0]:
federal = df.query('FederallyFunded == "Yes"')['GraduationPercentageRate']
unfederal = df.query('FederallyFunded == "No"')['GraduationPercentageRate']

In [0]:
stats.ttest_ind(federal,unfederal)

Out[10]: Ttest_indResult(statistic=-6.247729309320283, pvalue=1.9963190083589704e-09)

In [0]:
federallyfundeddf = df.loc[df['FederallyFunded'] == 'Yes']
federallyfundeddf.describe()

Out[11]:

,GraduationPercentageRate,DropoutPercentageRate,FundingPerStudent,TotalEnrolled,MalesEnrolled,FemalesEnrolled,Minority,HouseholdIncome
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,97.000000
mean,83.360000,6.360000,21122.053500,11889.160000,6140.020000,5749.140000,50.450000,68754.330515
std,9.354435,4.113344,13926.396181,14371.997423,7398.225362,7029.949726,33.848944,22728.801726
min,52.000000,0.000000,3298.420000,716.000000,338.000000,378.000000,3.400000,31104.500000
25%,76.000000,3.000000,17095.312500,2411.250000,1215.500000,1145.500000,18.050000,53729.000000
50%,85.000000,5.000000,18872.500000,4514.000000,2357.500000,2161.500000,48.900000,62700.290000
75%,91.000000,8.000000,21507.700000,19013.000000,9612.250000,9111.000000,84.550000,77814.000000
max,98.000000,19.000000,111293.330000,59568.000000,29605.000000,31112.000000,99.900000,146793.250000


In [0]:
unfederallyfundeddf = df.loc[df['FederallyFunded'] == 'No']
unfederallyfundeddf.describe()

Out[12]:

,GraduationPercentageRate,DropoutPercentageRate,FundingPerStudent,TotalEnrolled,MalesEnrolled,FemalesEnrolled,Minority,HouseholdIncome
count,132.000000,132.000000,132.000000,132.000000,132.000000,132.000000,132.000000,131.000000
mean,90.227273,3.765152,21962.976515,4104.393939,2103.121212,2001.272727,35.712121,95075.407481
std,7.386382,3.777919,4817.365709,2982.877019,1528.235594,1455.769632,26.622842,32240.392194
min,56.000000,0.000000,13662.020000,880.000000,451.000000,429.000000,1.700000,42558.000000
25%,87.750000,1.000000,18295.815000,2127.000000,1094.250000,1033.500000,14.900000,65841.885000
50%,92.000000,3.000000,21231.570000,3233.000000,1677.000000,1554.500000,28.600000,89395.400000
75%,95.000000,6.000000,25410.607500,5336.750000,2738.750000,2576.500000,44.350000,122678.355000
max,99.000000,19.000000,38615.960000,25017.000000,12904.000000,12113.000000,99.000000,171975.000000


Funding Per Student T-Test Analysis

In [0]:
stdover = df.query('FundingPerStudent > 32872')['GraduationPercentageRate']
stdunder = df.query('FundingPerStudent < 13630')['GraduationPercentageRate']
regulars = df.query('FundingPerStudent < 32872 and FundingPerStudent > 13630')['GraduationPercentageRate']

In [0]:
stats.ttest_ind(stdover,regulars)

Out[14]: Ttest_indResult(statistic=-0.8453621691354589, pvalue=0.3981983416576719)

In [0]:
stats.ttest_ind(stdunder,regulars)

Out[15]: Ttest_indResult(statistic=-3.2130285709636697, pvalue=0.001377042134524941)

In [0]:
highestfundeddistricts = df.loc[df['FundingPerStudent'] > 32872]
highestfundeddistricts.describe()

Out[27]:

,GraduationPercentageRate,DropoutPercentageRate,FundingPerStudent,TotalEnrolled,MalesEnrolled,FemalesEnrolled,Minority,HouseholdIncome
count,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,47.000000
mean,87.571429,3.979592,48404.679184,1032.102041,541.387755,490.714286,32.085714,89658.304681
std,14.997222,5.987799,20396.589501,1955.385259,1037.198105,920.216732,28.457285,31755.773367
min,22.000000,0.000000,33287.480000,13.000000,5.000000,0.000000,1.500000,38831.500000
25%,87.000000,0.000000,35628.810000,71.000000,39.000000,36.000000,8.300000,58844.835000
50%,91.000000,2.000000,38857.670000,207.000000,109.000000,100.000000,25.900000,92963.340000
75%,96.000000,5.000000,53804.950000,1046.000000,526.000000,523.000000,37.700000,114601.330000
max,100.000000,33.000000,117584.610000,9850.000000,5045.000000,4805.000000,98.700000,152512.580000


In [0]:
lowestfundeddistricts = df.loc[df['FundingPerStudent'] < 13630]
lowestfundeddistricts.describe()

Out[26]:

,GraduationPercentageRate,DropoutPercentageRate,FundingPerStudent,TotalEnrolled,MalesEnrolled,FemalesEnrolled,Minority,HouseholdIncome
count,14.000000,14.000000,14.000000,14.000000,14.000000,14.00000,14.000000,13.000000
mean,81.285714,6.142857,9386.046429,27535.928571,14570.928571,12965.00000,66.907143,68659.110000
std,7.268780,1.791310,2920.750528,21163.595748,11023.138681,10359.47965,33.340561,15888.345939
min,71.000000,4.000000,3298.420000,409.000000,205.000000,204.00000,6.000000,39264.330000
25%,76.000000,5.000000,7941.292500,5812.250000,2873.500000,2223.00000,59.175000,62593.000000
50%,79.500000,6.000000,9999.530000,27602.000000,16710.000000,12311.50000,79.450000,70628.000000
75%,85.500000,7.000000,10875.440000,47204.500000,24227.000000,22547.00000,89.325000,74852.770000
max,93.000000,10.000000,13567.300000,57787.000000,29605.000000,28182.00000,97.900000,95562.130000


Districts with Minority Percentage > 60%

In [0]:
overminority = df.query(' Minority > 60')['GraduationPercentageRate']
regularminority = df.query('Minority < 60')['GraduationPercentageRate']

In [0]:
stats.ttest_ind(overminority,regularminority)

Out[19]: Ttest_indResult(statistic=-11.681999482727674, pvalue=5.766124662581394e-29)

In [0]:
minorityoverdf = df.loc[df['Minority'] > 60] 
minorityunderdf = df.loc[df['Minority'] < 60] 

In [0]:
minorityunderdf.describe()

Out[21]:

,GraduationPercentageRate,DropoutPercentageRate,FundingPerStudent,TotalEnrolled,MalesEnrolled,FemalesEnrolled,Minority,HouseholdIncome
count,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,609.000000,585.000000
mean,90.031199,4.338259,23066.290805,1954.374384,999.090312,955.284072,17.470115,76361.012427
std,6.966604,4.496768,8461.656995,2950.648343,1509.945700,1441.084333,14.430204,27481.053645
min,40.000000,0.000000,5403.880000,13.000000,5.000000,7.000000,0.000000,42558.000000
25%,87.000000,1.000000,18437.890000,639.000000,329.000000,312.000000,6.000000,56750.000000
50%,91.000000,3.000000,21091.380000,1184.000000,602.000000,580.000000,11.900000,65364.750000
75%,95.000000,6.000000,25303.290000,2490.000000,1308.000000,1244.000000,26.900000,89023.660000
max,100.000000,53.000000,117584.610000,57787.000000,29605.000000,28182.000000,59.900000,171975.000000


In [0]:
minorityoverdf.describe()

Out[22]:

,GraduationPercentageRate,DropoutPercentageRate,FundingPerStudent,TotalEnrolled,MalesEnrolled,FemalesEnrolled,Minority,HouseholdIncome
count,104.000000,104.000000,104.000000,104.000000,104.000000,104.000000,104.000000,91.000000
mean,79.586538,7.307692,24327.020577,11815.932692,6110.000000,5705.932692,82.100000,87733.820769
std,14.271712,5.819413,14723.636080,13428.825087,6913.568773,6573.050295,12.441568,32854.527043
min,9.000000,0.000000,3298.420000,29.000000,14.000000,0.000000,60.300000,31104.500000
25%,75.000000,3.750000,18849.085000,2623.000000,1338.500000,1278.500000,72.025000,56653.905000
50%,81.500000,6.000000,21740.005000,6210.500000,3143.500000,3020.500000,83.400000,91574.840000
75%,89.250000,10.000000,25084.052500,19013.000000,9612.250000,9111.000000,94.400000,110825.830000
max,98.000000,33.000000,111293.330000,59568.000000,28456.000000,31112.000000,99.900000,160936.330000
